# DemoStrategy

## 视频中介绍了针对四只股票的等权重投资策略，本段代码展示了利用quantOS系统进行策略回测及仿真交易的具体步骤。
## 在程序运行之前，需要您在环境设置中更改以下参数：
## 1. 将YourPhoneNo.改为您在quantOS网站注册的手机号;
## 2. 将YourToken改为您的token;
## 3. 将YourStrategyNo.改为您的策略号。

## 1. 环境设置

In [1]:
# -*- encoding: utf-8 -*-
import time
import pandas as pd
import numpy as np

from jaqs.data import RemoteDataService
from jaqs.trade import AlphaBacktestInstance
from jaqs.trade import PortfolioManager
#from jaqs.trade import RealTimeTradeApi

import jaqs.util as jutil
import jaqs.trade.analyze as ana
from jaqs.trade import AlphaStrategy
from jaqs.trade import AlphaTradeApi
from jaqs.trade import model
from jaqs.data import DataView

In [6]:
# 设置文件存储路径
dataview_dir_path = 'demoStrategy/dataview'
backtest_result_dir_path = 'demoStrategy'

# 设置服务器地址、用户名密码
# 例如：
# data_config = {
#   "remote.data.address": "tcp://data.tushare.org:8910",
#   "remote.data.username":  '18688888888',
#   "remote.data.password":  '23sdjfk209d0fs9dejkl2j3k4j9d0fsdf'}

phone = "xxxx"
token = "xxxxxxxxxx"

data_config = {
  "remote.data.address": "tcp://data.tushare.org:8910",
  "remote.data.username":  phone,
  "remote.data.password":  token}
trade_config = {
  "remote.trade.address": "tcp://gw.quantos.org:8901",
  "remote.trade.username":  phone,
  "remote.trade.password":  token}


# 设置Strategy number
# 例如：StrategyNo = 1043
StrategyNo = 'YourStrategyNo.'

## 2. 参数设置

In [7]:
# -------------------------------------------------------------------------------
# 设置目标股票、业绩基准、权重、时间
# -------------------------------------------------------------------------------
symbol_weights = {'600519.SH': 0.25,
                  '600036.SH': 0.25,
                  '601318.SH': 0.25,
                  '000651.SZ': 0.25}

benchmark = '000300.SH'

my_symbols = ','.join(symbol_weights.keys())
start_date = 20170201
end_date = 20171001

# 定义权重函数
def stockWeight(context, user_options=None):
    return pd.Series(symbol_weights)

## 3. 回测

In [8]:
# -------------------------------------------------------------------------------
# Main code 这个代码框不需要修改
# -------------------------------------------------------------------------------

def test_save_dataview():
    ds = RemoteDataService()
    ds.init_from_config(data_config)
    dv = DataView()

    props = {'start_date': start_date, 'end_date': end_date,
             'fields': 'sw1',
             'symbol': my_symbols,
             'freq': 1}

    dv.init_from_config(props, ds)
    dv.prepare_data()

    # set the benchmark
    res, _ = ds.daily(benchmark, start_date=dv.start_date, end_date=dv.end_date)
    dv._data_benchmark = res.set_index('trade_date').loc[:, ['close']]

    dv.save_dataview(folder_path=dataview_dir_path)


def test_alpha_strategy_dataview():
    dv = DataView()

    dv.load_dataview(folder_path=dataview_dir_path)

    props = {
        "symbol": dv.symbol,
        "universe": ','.join(dv.symbol),

        "start_date": dv.start_date,
        "end_date": dv.end_date,

        "period": "week",
        "days_delay": 0,

        "init_balance": 1e7,
        "position_ratio": 1.0,
        "commission_rate": 2E-4  # 手续费万2
    }
    props.update(data_config)
    props.update(trade_config)

    trade_api = AlphaTradeApi()

    signal_model = model.FactorSignalModel()
    signal_model.add_signal('stockWeight', stockWeight)

    strategy = AlphaStrategy(signal_model=signal_model, pc_method='factor_value_weight')
    pm = PortfolioManager()

    bt = AlphaBacktestInstance()
    
    context = model.Context(dataview=dv, instance=bt, strategy=strategy, trade_api=trade_api, pm=pm)
    
    signal_model.register_context(context)

    bt.init_from_config(props)

    bt.run_alpha()

    bt.save_results(folder_path=backtest_result_dir_path)
    

def test_backtest_analyze():
    ta = ana.AlphaAnalyzer()
    dv = DataView()
    dv.load_dataview(folder_path=dataview_dir_path)

    ta.initialize(dataview=dv, file_folder=backtest_result_dir_path)

    ta.do_analyze(result_dir=backtest_result_dir_path, selected_sec=ta.universe,
                  brinson_group=None)


In [9]:
# 运行这里跑回测
test_save_dataview()
test_alpha_strategy_dataview()
test_backtest_analyze()



Begin: DataApi login 18612562791@tcp://data.tushare.org:8910
    login success 

Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query groups (industry)...
Data has been successfully prepared.

Store data...
Dataview has been successfully saved to:
C:\Users\jfang\ipython\demoStrategy\dataview

You can load it with load_dataview('C:\Users\jfang\ipython\demoStrategy\dataview')
Dataview loaded successfully.
AlphaStrategy Initialized.

=======new day 20170203
Before 20170203 re-balance: available cash all (exclude suspensions) = 1.0000e+07

=======new day 20170206
Before 20170206 re-balance: available cash all (exclude suspensions) = 9.9970e+06

=======new day 20170213
Before 20170213 re-balance: available cash all (exclude suspensions) = 1.0272e+07

=======new day 20170220
Before 20170220 re-balance: available cash all (exclude suspensions) = 1.0588e+

回测显示运行完成后，报告可从上面对话框最后一行的地址中找到 `HTML report: ...\demoStrategy\report.html`

## 4. 仿真交易

In [10]:
from jaqs.trade.tradeapi import TradeApi

In [11]:
tapi = TradeApi(trade_config['remote.trade.address'])

Exception No module named 'jrpc'
TradeApi: _on_connected


In [13]:
def on_orderstatus(order):
    print("on_orderstatus:") #, order
    for key in order:    print("%20s : %s" % (key, str(order[key])))
    print("")


# 成交回报推送
def on_trade(trade):
    print("on_trade:")
    for key in trade:    print("%20s : %s" % (key, str(trade[key])))
    print("")

# 委托任务执行状态推送
# 通常可以忽略该回调函数
def on_taskstatus(task):
    print("on_taskstatus:")
    for key in task:    print("%20s : %s" % (key, str(task[key])))
    print("")

tapi.set_ordstatus_callback(on_orderstatus)
tapi.set_trade_callback(on_trade)
tapi.set_task_callback(on_taskstatus)

In [14]:
user_info, msg = tapi.login(trade_config['remote.trade.username'], trade_config['remote.trade.password'])

In [16]:
print(user_info)
tapi.use_strategy(StrategyNo)  #改成用户自己的 strategy号

res, msg = tapi.query_account()
print(res)

money = res.loc[0, 'enable_balance']
print("Balance we have: {}".format(money))

{'username': '18612562791', 'name': 'jfang', 'strategies': [1008, 1009, 1010], 'broker_strategies': []}
   close_pnl  deposit_balance  enable_balance  float_pnl  frozen_balance  \
0        0.0              0.0     100000000.0   -55544.0             0.0   
1        0.0              0.0     100000000.0   -55544.0             0.0   

   holding_pnl    id  init_balance  margin  trading_pnl type  
0      52580.0  1008   100000000.0     0.0          0.0   SA  
1      52580.0  1008   100000000.0     0.0          0.0   BA  
Balance we have: 100000000.0


In [17]:
dic_price      = {'600519.SH': 637.70,
                  '600036.SH': 28.72,
                  '601318.SH': 63.29,
                  '000651.SZ': 41.96}

# 每只股票等金额投资，各买入30万元
dic_shares = {k: 100 * np.floor(money * 0.012 * symbol_weights[k] / dic_price[k]/100) for k, _ in symbol_weights.items()}
dic_shares

orders = []
for symbol in symbol_weights.keys():
    o = {'security': symbol, 'price': dic_price[symbol], 'size': dic_shares[symbol], 'action': 'Buy'}
    orders.append(o)
orders

[{'action': 'Buy', 'price': 637.7, 'security': '600519.SH', 'size': 400.0},
 {'action': 'Buy', 'price': 28.72, 'security': '600036.SH', 'size': 10400.0},
 {'action': 'Buy', 'price': 63.29, 'security': '601318.SH', 'size': 4700.0},
 {'action': 'Buy', 'price': 41.96, 'security': '000651.SZ', 'size': 7100.0}]

In [19]:
# 买入股票
task_id, msg = tapi.place_batch_order(orders)
print(task_id)
print(msg)    

10081211000001
0,scheduled in 142837215 ms
on_orderstatus:
               sa_id : 1008
               ba_id : 100800
          entrust_no : 10081211000001
            security : 600519.SH
      entrust_action : Buy
       entrust_price : 637.7
        entrust_size : 400
        entrust_date : 20171209
        entrust_time : 174922
             ord_seq : 0
            batch_no : 0
        order_status : New
         is_finished : False
          fill_price : 0.0
           fill_size : 0
          commission : 0.0
             task_id : 10081211000001
                algo : 

on_orderstatus:
               sa_id : 1008
               ba_id : 100800
          entrust_no : 10081211000002
            security : 600036.SH
      entrust_action : Buy
       entrust_price : 28.72
        entrust_size : 10400
        entrust_date : 20171209
        entrust_time : 174922
             ord_seq : 1
            batch_no : 0
        order_status : New
         is_finished : False
          fill_price 

In [20]:
# 做空一手沪深300股指期货做对冲
# task_id, msg = tapi.place_order("IF1712.CFE", "Short", 4003.6, 1)
# print(task_id)
# print(msg)

0
0,IF1712.CFE not in UNIVERSE
